In [5]:
import numpy as np

class CustomLinearRegression:
    def __init__(self, X_data, y_target, learning_rate=0.01, num_epochs=10000):
        self.num_samples = X_data.shape[0]
        self.X_data = np.c_[np.ones((self.num_samples, 1)), X_data]
        self.y_target = y_target.reshape(-1, 1) 
        self.learning_rate = learning_rate
        self.num_epochs = num_epochs

        # Initial weights (random initialization)
        self.theta = np.random.randn(self.X_data.shape[1], 1)
        self.losses = []

    def compute_loss(self, y_pred, y_target):
        loss = (1 / (2 * self.num_samples)) * np.sum((y_pred - y_target) ** 2)
        return loss

    def predict(self, X_data):
        num_samples = X_data.shape[0]
        X_data_b = np.c_[np.ones((num_samples, 1)), X_data]  # Add bias term
        y_pred = X_data_b.dot(self.theta)
        return y_pred

    def fit(self):
        for epoch in range(self.num_epochs):
            # Compute predictions
            y_pred = self.X_data.dot(self.theta)

            # Compute loss
            loss = self.compute_loss(y_pred, self.y_target)
            self.losses.append(loss)

            # Compute gradient
            gradient = (1 / self.num_samples) * self.X_data.T.dot(y_pred - self.y_target)

            # Update weights
            self.theta -= self.learning_rate * gradient

            # Print loss every 50 epochs
            if (epoch % 50) == 0:
                print(f'Epoch: {epoch} - Loss: {loss}')

        return {
            'loss': sum(self.losses) / len(self.losses),
            'weight': self.theta
        }


In [6]:
import numpy as np

def create_polynomial_features(X, degree=2):
    """ Creates the polynomial features
    Args:
        X : A array tensor for the data.
        degree : An integer for the degree of the generated polynomial function.
    """
    X_new = X
    for d in range(2, degree + 1):
        X_new = np.c_[X_new, np.power(X, d)]
    return X_new

# Ví dụ sử dụng
X = np.array([[1], [2], [3]])
degree = 2
X_poly = create_polynomial_features(X, degree)
print(X_poly)


[[1 1]
 [2 4]
 [3 9]]


In [ ]:
def create_polynomial_features(X, degree=2):
    """ Creates the polynomial features
    Args:
        X : A array for the data.
        degree : An integer for the degree of the generated polynomial function.
    """
    X_mem = []
    for X_sub in X.T:
        X_sub = X_sub.T
        X_new = X_sub
        for d in range(2, degree + 1):
            X_new = np.c_[X_sub, np.power(X_sub, d)]
        X_mem.extend(X_new.T)
    return np.c_[X_mem].T


## Sales Prediction

In [7]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

# Load dataset
df = pd.read_csv('SalesPrediction.csv')

# Preprocessing
df = pd.get_dummies(df)  # One-hot encoding for 'Influencer' column
df = df.fillna(df.mean())  # Handle Null values

# Get features and target
X = df[['TV', 'Radio', 'Social Media', 'Influencer_Macro', 'Influencer_Mega', 'Influencer_Micro', 'Influencer_Nano']]
y = df[['Sales']]

# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0)

# Feature Scaling
scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train)
X_test_processed = scaler.transform(X_test)

# Check the mean of the first feature after scaling
mean_first_feature = scaler.mean_[0]

# Polynomial Features
poly_features = PolynomialFeatures(degree=2)
X_train_poly = poly_features.fit_transform(X_train_processed)
X_test_poly = poly_features.transform(X_test_processed)

# Training & Evaluation
poly_model = LinearRegression()
poly_model.fit(X_train_poly, y_train)
preds = poly_model.predict(X_test_poly)
r2 = r2_score(y_test, preds)

# Output the mean of the first feature after scaling
print(mean_first_feature)
# Output the R² score
print(r2)

54.173577723283785
0.9951495108621895
